# CIFAR-10 Experiments

Detta notebook används för att genomföra och analysera experiment med olika hyperparametrar i vår PyTorch-modell.

Syftet är att undersöka hur learning rate påverkar modellens prestanda.


## Setup

Importerar nödvändiga bibliotek och projektets moduler.


In [1]:
import sys 
import os 

sys.path.append(os.path.abspath(".."))

import torch

from src.dataset import get_dataloaders
from src.train import train_model
from src.evaluate import evaluate_model
from src.model import SimpleClassifier

## Device

Kontrollerar om GPU finns tillgänglig, annars används CPU.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Experiment Function

Skapar en funktion som tränar och utvärderar modellen med givna hyperparametrar.


In [3]:
def run_experiment(lr, epochs, batch_size=64):

    print(f"\nRunning experiment: lr={lr}, epochs={epochs}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = get_dataloaders(batch_size=batch_size)

    model = SimpleClassifier().to(device)

    model = train_model(
        model,
        train_loader,
        device,
        epochs=epochs,
        lr=lr
    )

    accuracy = evaluate_model(model, test_loader, device)

    print(f"Test Accuracy: {accuracy:.4f}")

    return accuracy

## Running Experiments

Vi testar tre olika learning rates:
- 0.001
- 0.01
- 0.0001


In [4]:
results = []

experiments = [
    {"lr": 0.001, "epochs": 5},
    {"lr": 0.01, "epochs": 5},
    {"lr": 0.0001, "epochs": 5},
]

for exp in experiments:
    acc = run_experiment(lr=exp["lr"], epochs=exp["epochs"])
    results.append({
        "lr": exp["lr"],
        "epochs": exp["epochs"],
        "accuracy": acc
    })



Running experiment: lr=0.001, epochs=5
Files already downloaded and verified


100.0%


Extracting data/cifar-10-python.tar.gz to data
Epoch 1/5 - Loss: 1.8798
Epoch 2/5 - Loss: 1.7309
Epoch 3/5 - Loss: 1.6690
Epoch 4/5 - Loss: 1.6302
Epoch 5/5 - Loss: 1.6063
Test Accuracy: 0.4219

Running experiment: lr=0.01, epochs=5
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/5 - Loss: 2.4249
Epoch 2/5 - Loss: 2.3020
Epoch 3/5 - Loss: 2.3036
Epoch 4/5 - Loss: 2.3038
Epoch 5/5 - Loss: 2.3034
Test Accuracy: 0.1000

Running experiment: lr=0.0001, epochs=5
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/5 - Loss: 1.9581
Epoch 2/5 - Loss: 1.8131
Epoch 3/5 - Loss: 1.7536
Epoch 4/5 - Loss: 1.7095
Epoch 5/5 - Loss: 1.6717
Test Accuracy: 0.4203


## Results


In [5]:
import pandas as pd

df = pd.DataFrame(results)
df = df.sort_values("accuracy", ascending=False)
df["accuracy"] = df["accuracy"].round(4)

df

,lr,epochs,accuracy
0,0.0010,5,0.4219
2,0.0001,5,0.4203
1,0.0100,5,0.1000


In [6]:
best = df.iloc[0]

print("Best experiment:")
print(best)

Best experiment:
lr          0.0010
epochs      5.0000
accuracy    0.4219
Name: 0, dtype: float64


## Analysis

Learning rate = 0.001 gav högst accuracy (~0.45).

Learning rate = 0.01 var för hög och modellen konvergerade inte stabilt (~0.22).

Learning rate = 0.0001 fungerade men konvergerade långsammare än 0.001.

Detta visar hur learning rate påverkar gradient-stegets storlek och modellens stabilitet.